In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/HMDB_simp.zip -d /content/HMDB_simp

Streaming output truncated to the last 5000 lines.
  inflating: /content/HMDB_simp/HMDB_simp/smile/F11051A0/0036.jpg  
  inflating: /content/HMDB_simp/__MACOSX/HMDB_simp/smile/F11051A0/._0036.jpg  
  inflating: /content/HMDB_simp/HMDB_simp/smile/F11051A0/0022.jpg  
  inflating: /content/HMDB_simp/__MACOSX/HMDB_simp/smile/F11051A0/._0022.jpg  
  inflating: /content/HMDB_simp/HMDB_simp/smile/F11051A0/0023.jpg  
  inflating: /content/HMDB_simp/__MACOSX/HMDB_simp/smile/F11051A0/._0023.jpg  
  inflating: /content/HMDB_simp/HMDB_simp/smile/F11051A0/0037.jpg  
  inflating: /content/HMDB_simp/__MACOSX/HMDB_simp/smile/F11051A0/._0037.jpg  
  inflating: /content/HMDB_simp/HMDB_simp/smile/F11051A0/0044.jpg  
  inflating: /content/HMDB_simp/__MACOSX/HMDB_simp/smile/F11051A0/._0044.jpg  
  inflating: /content/HMDB_simp/HMDB_simp/smile/F11051A0/0045.jpg  
  inflating: /content/HMDB_simp/__MACOSX/HMDB_simp/smile/F11051A0/._0045.jpg  
  inflating: /content/HMDB_simp/HMDB_simp/smile/F11051A0/0047.jpg  

In [ ]:
!pip install transformers torch torchvision datasets evaluate torchmetrics

# Data

## Create dataset

In [1]:
CATEGORY_INDEX = {
    "brush_hair": 0,
    "cartwheel": 1,
    "catch": 2,
    "chew": 3,
    "climb": 4,
    "climb_stairs": 5,
    "draw_sword": 6,
    "eat": 7,
    "fencing": 8,
    "flic_flac": 9,
    "golf": 10,
    "handstand": 11,
    "kiss": 12,
    "pick": 13,
    "pour": 14,
    "pullup": 15,
    "pushup": 16,
    "ride_bike": 17,
    "shoot_bow": 18,
    "shoot_gun": 19,
    "situp": 20,
    "smile": 21,
    "smoke": 22,
    "throw": 23,
    "wave": 24
}

In [5]:
import os
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm.notebook import trange, tqdm

# Transformation: Resize to 224x224 and Convert to Tensor
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

def load_sampled_frames(frame_dir, frame_rate=8):
    """
    Load every [frame_rate]-th frame from a directory and apply transformations.
    """
    frame_files = sorted(os.listdir(frame_dir))  # Ensure frames are in order
    sampled_frames = []

    for i in range(0, len(frame_files), frame_rate):
        frame_path = os.path.join(frame_dir, frame_files[i])
        frame = Image.open(frame_path).convert("RGB")  # Convert to RGB
        frame = transform(frame)  # Apply transformations
        sampled_frames.append(frame)

    return sampled_frames  # List of torch tensors

def create_clips(frames, clip_size=8):
    """
    Given a list of sampled frames, create multiple [clip_size]-frame clips.
    """
    clips = []
    if len(frames) < clip_size:
        return clips  # Not enough frames to create a clip
    for i in range(0, len(frames) - clip_size + 1, clip_size):  # Sliding window
        clip = torch.stack(frames[i:i + clip_size])  # Stack into (clip_size, 3, 224, 224)
        clips.append(clip)

    return clips

In [6]:
DATASET_PATH = "/content/HMDB_simp/HMDB_simp"

import random

def split_sources(dataset_path, train_ratio=0.8):
    """
    Splits source folders into train and val sets before processing clips.
    Ensures that all clips from a source video stay in the same set.
    """
    train_sources = {}
    val_sources = {}

    for category in os.listdir(dataset_path):  # Iterate over action categories
        category_path = os.path.join(dataset_path, category)
        if not os.path.isdir(category_path):
            continue

        instances = os.listdir(category_path)  # List all source folders (video IDs)
        random.shuffle(instances)  # Shuffle instances before splitting

        split_idx = int(len(instances) * train_ratio)
        train_sources[category] = instances[:split_idx]  # First 80% for training
        val_sources[category] = instances[split_idx:]  # Last 20% for validation

    return train_sources, val_sources


def process_dataset(dataset_path, sources_dict):
    """
    Processes dataset based on a predefined list of sources.
    """
    dataset = []

    for category, instances in tqdm(sources_dict.items()):
        category_path = os.path.join(dataset_path, category)

        for instance in instances:
            instance_path = os.path.join(category_path, instance)
            if not os.path.isdir(instance_path):
                continue  # Skip non-directory files

            # Load sampled frames
            frames = load_sampled_frames(instance_path)

            # Create 8-frame clips
            clips = create_clips(frames)

            for clip in clips:
                dataset.append((clip, CATEGORY_INDEX[category]))  # Store (clip, label)

    return dataset  # List of (clip, label)

class VideoDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        clip, label = self.dataset[idx]
        return clip, torch.tensor(label, dtype=torch.long)

from torch.utils.data import default_collate

class VideoDataCollator:
    """
    Custom data collator for TimeSFormer.
    Converts (clip, label) tuples into a dictionary format.
    """
    def __call__(self, features):
        clips, labels = zip(*features)  # Unpack (clip, label)
        batch = {
            "pixel_values": torch.stack(clips),  # Stack clips into batch
            "labels": torch.tensor(labels, dtype=torch.long)  # Convert labels to tensor
        }
        return batch


In [7]:
# Split source folders into train & val
train_sources, val_sources = split_sources(DATASET_PATH)

# Process train and val sets separately
train_dataset = process_dataset(DATASET_PATH, train_sources)
val_dataset = process_dataset(DATASET_PATH, val_sources)

dataset_size = len(train_dataset) + len(val_dataset)

print(f"Total clips: {dataset_size}, Train: {len(train_dataset)}, Val: {len(val_dataset)}")

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

Total clips: 1627, Train: 1318, Val: 309


# Train

## TimeSFormer

In [9]:
from transformers import AutoFeatureExtractor, AutoModelForVideoClassification


extractor = AutoFeatureExtractor.from_pretrained("facebook/timesformer-base-finetuned-k400")
model = AutoModelForVideoClassification.from_pretrained(
    "facebook/timesformer-base-finetuned-k400",
    num_labels=len(CATEGORY_INDEX),  # Adjust for our dataset
    ignore_mismatched_sizes=True,
)

# Send model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Model on: {device}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/videomae/feature_extraction_videomae.py:28: FutureWarning: The class VideoMAEFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use VideoMAEImageProcessor instead.
  warnings.warn(
Some weights of TimesformerForVideoClassification were not initialized from the model checkpoint at facebook/timesformer-base-finetuned-k400 and are newly initialized because the shapes did not match:
- c

Model on: cuda


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchmetrics
import evaluate
import numpy as np
from transformers import AutoFeatureExtractor, AutoModelForVideoClassification, TrainingArguments, Trainer

accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
top5_metric = torchmetrics.classification.Accuracy(top_k=5, task="multiclass", num_classes=len(CATEGORY_INDEX)).to(device)

# Compute Metrics Function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.tensor(logits).argmax(dim=1)

    # Compute Accuracy
    top1_acc = accuracy_metric.compute(predictions=predictions.numpy(), references=labels)["accuracy"]

    # Compute Top-5 Accuracy
    top5_acc = top5_metric(torch.tensor(logits).to(device), torch.tensor(labels).to(device)).item()

    # Compute F1-score (macro)
    f1 = f1_metric.compute(predictions=predictions.numpy(), references=labels, average="macro")["f1"]

    return {
        "accuracy": top1_acc,
        "top-5 accuracy": top5_acc,
        "f1-score": f1
    }

In [11]:
training_args = TrainingArguments(
    output_dir="./timesformer_output",  # Save checkpoints
    evaluation_strategy="epoch",  # Evaluate after every epoch
    save_strategy="epoch",  # Save model after each epoch
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="./logs",  # TensorBoard logs
    logging_steps=10,
    save_total_limit=2,  # Keep only last 2 checkpoints
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False
)

data_collator = VideoDataCollator()
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=extractor,  # Feature extractor
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

# Train Model
trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-11-14e7e5a34a30>:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: karkartem to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Top-5 accuracy,F1-score
1,0.508000,0.645068,0.825243,0.970874,0.790915
2,0.076700,0.595146,0.844660,0.977346,0.828910
3,0.001500,0.537473,0.857605,0.980583,0.852075
4,0.001300,0.499854,0.873786,0.977346,0.857364
5,0.000800,0.493607,0.877023,0.977346,0.858724


TrainOutput(global_step=1650, training_loss=0.23152913113689105, metrics={'train_runtime': 3881.9868, 'train_samples_per_second': 1.698, 'train_steps_per_second': 0.425, 'total_flos': 5.77465293172949e+18, 'train_loss': 0.23152913113689105, 'epoch': 5.0})